  # Data Cleaning

The purpose of this activity is to impute data for variables that will be for model building and also resolve any data quality issues we observed during exploration

In [289]:
import pandas as pd 
import numpy as np 
import os
import math

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from wordcloud import WordCloud
from wordcloud import STOPWORDS
import nltk


## Load Data

In [290]:
path = "."
fname = os.path.join(path, "raw_data.csv")
df = pd.read_csv(fname)

In [291]:
columns=['company', 'timestamp','title', 'totalyearlycompensation', 'location', 'yearsofexperience', 'yearsatcompany', 
         'tag','gender', 'Race', 'Education', 'rowNumber', 'otherdetails', 'basesalary', 'stockgrantvalue'
         , 'bonus']
df = df[columns]

In [292]:
display(df.head())

,company,timestamp,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,gender,Race,Education,rowNumber,otherdetails,basesalary,stockgrantvalue,bonus
0,Oracle,6/7/2017 11:33:27,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,NaN,NaN,NaN,1,NaN,107000.0,20000.0,10000.0
1,eBay,6/10/2017 17:11:29,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,NaN,NaN,NaN,2,NaN,0.0,0.0,0.0
2,Amazon,6/11/2017 14:53:57,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,NaN,NaN,NaN,3,NaN,155000.0,0.0,0.0
3,Apple,6/17/2017 0:23:14,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,NaN,NaN,NaN,7,NaN,157000.0,180000.0,35000.0
4,Microsoft,6/20/2017 10:58:51,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,NaN,NaN,NaN,9,NaN,0.0,0.0,0.0


## Clean Company

In [293]:
t = df.groupby(['company']).count().sort_values(by=['timestamp'], ascending = False).rename(columns={'timestamp':'count'}).reset_index() #data clean up 
t[['company', 'count']]
print("Before cleaning: {}".format(len(t.company)))

Before cleaning: 1631


In [294]:
t['company_clean'] = t.company.str.rstrip().str.lstrip().str.upper()
print("After cleaning: {}".format(len(t.company_clean.unique()))) 

After cleaning: 1100


In [295]:
#After cleaning see how salaries are reported across companies 
df['company_clean'] = df.company.str.rstrip().str.lstrip().str.upper()
s = df.groupby(['company_clean']).count().sort_values(by=['timestamp'], ascending = False).rename(columns={'timestamp':'count_'}).reset_index() #data clean up 
s = s[['company_clean', 'count_']] # need data cleaning?
s['cum_count'] = s['count_'].cumsum()
s['cum_perc'] = (s['cum_count']/s.count_.sum())*100
s[s.cum_perc <= 50.0]  #see how many companies it takes to capture 50% of data 

,company_clean,count_,cum_count,cum_perc
0,AMAZON,8199,8199,13.089707
1,MICROSOFT,5241,13440,21.456966
2,GOOGLE,4363,17803,28.422498
3,FACEBOOK,3004,20807,33.218385
4,APPLE,2047,22854,36.486422
5,ORACLE,1143,23997,38.311222
6,SALESFORCE,1065,25062,40.011495
7,INTEL,966,26028,41.553714
8,IBM,927,26955,43.033670
9,CISCO,923,27878,44.507240


In [296]:
print("Num of companies with 95% data:", s[s.cum_perc <= 95.0].count()[0]) 
print("Num of companies with 90% data:", s[s.cum_perc <= 90.0].count()[0])
print("Num of companies with 80% of data:", s[s.cum_perc <= 80.0].count()[0])
print("Num of companies with 70% data:", s[s.cum_perc <= 70.0].count()[0])
print("Num of companies with 50% data:", s[s.cum_perc <= 50.0].count()[0]) 

Num of companies with 95% data: 599
Num of companies with 90% data: 343
Num of companies with 80% of data: 137
Num of companies with 70% data: 63
Num of companies with 50% data: 14


In [297]:
company_names = t.company_clean.unique()

#try levenshtein to see if we can clean up the names more 
cols = ["Company", "ClosestMatch", "Score"]
df_companyClosestMatch = pd.DataFrame(columns = cols)

for company in company_names:
    max_score = 0
    closestMatch = None
    #company_name = None 
    for val_to_compare in company_names:
        if val_to_compare == company:
            continue 
        else:
            score =  fuzz.token_sort_ratio(company, val_to_compare)
            if score > max_score:
                max_score = score 
                closestMatch = val_to_compare
    df_companyClosestMatch = df_companyClosestMatch.append(pd.DataFrame({"Company": [company],
                                                 "ClosestMatch": [closestMatch],
                                                 "Score": [max_score]
                                                   }))


In [298]:
df_companyClosestMatch # can't pick a threshold score based on observed values 

,Company,ClosestMatch,Score
0,AMAZON,AMZON,91
0,MICROSOFT,MICRON,67
0,GOOGLE,GOOGLE LLC,75
0,FACEBOOK,FACTSET,53
0,APPLE,RIPPLE,73
...,...,...,...
0,XING,ING,86
0,VIZIO,VERIZON,67
0,IDEMIA,NVIDIA,67
0,QUEST DIAGNOSTICS,NEXTERA ANALYTICS,53


In [ ]:
# Try jacqquard distance to see if it yields better results 
# https://python.gotrained.com/nltk-edit-distance-jaccard-distance/ 

cols = ["Company", "ClosestMatch", "Score"]
df_companyClosestMatch = pd.DataFrame(columns = cols)

for company in company_names:
    min_score = 100
    closestMatch = None
    #company_name = None 
    for val_to_compare in company_names:
        if val_to_compare == company:
            continue 
        else:
            score = nltk.jaccard_distance(set(company), set(val_to_compare))
            if score < min_score:
                min_score = score 
                closestMatch = val_to_compare
    df_companyClosestMatch = df_companyClosestMatch.append(pd.DataFrame({"Company": [company],
                                                 "ClosestMatch": [closestMatch],
                                                 "Score": [min_score]
                                                   }))


In [ ]:
df_companyClosestMatch #no threshold score for which it makes sense to accept value

In [ ]:
df.drop(columns = ['company_clean'], inplace = True)

## Other details

In [ ]:
df['otherdetails'].dropna()

In [ ]:
#use word cloud to identify frequently occuring words 
my_additional_stop_words = ['Year', 'Tons', 'access']

stop_words = list(STOPWORDS) + my_additional_stop_words

wordcloud = WordCloud(stopwords = stop_words, background_color = 'white').generate(str(df['otherdetails'].dropna()))

print(wordcloud)
plt.rcParams['figure.figsize'] = (12, 12)
plt.axis('off')
plt.imshow(wordcloud)
plt.title('Most Occuring word in the Other Details', fontsize = 20)
plt.show()

In [ ]:
df.drop(columns = ['otherdetails'], inplace = True)

## Yearly compensation

In [ ]:
#data validation to see if all columns should be retained? 
salary_cols = [ 'totalyearlycompensation', 'basesalary',
 'stockgrantvalue',
 'bonus']
t = df[salary_cols].copy()#.info()
t['total'] = t['basesalary'] + t['stockgrantvalue'] + t['bonus']
t.loc[t.total != t.totalyearlycompensation].head()

In [ ]:
t['diff_perc'] = round(((t['totalyearlycompensation']-t['total'])/t['totalyearlycompensation'])*100)
t[['totalyearlycompensation', 'total', 'diff_perc' ]]

In [ ]:
def group_data(x):
    val = abs(x)
    if x <= 5: return "0-5"
    elif x<= 10: return "06-10"
    elif x<= 20: return "11-20"
    elif x<=40: return "21-40"
    elif x<=60: return "41-60"
    elif x<=80: return "61-80"
    else: return ">81"
    
t["perc_group"] = t["diff_perc"].apply(group_data)
t.groupby(['perc_group']).count() #use totalyearlycom? 

In [ ]:
t.loc[(t.diff_perc == 100) & (t.total != 0)] #breakdown not provided for all records 

In [ ]:
df.drop(columns = ['basesalary','stockgrantvalue', 'bonus'], inplace = True)

## Clean tags

In [ ]:
#find tag values that need to be mapped to current list
current_tag_list = ['iOS', 'Android', 'Mobile (iOS + Android)', 'Web Development (Front-End)', 'ML / AI', 'Distributed Systems (Back-End)', 'API Development (Back-End)', 'Full Stack',
'Networking', 'Testing (SDET)', 'Data', 'Production', 'Security', 'DevOps', 'Site Reliability (SRE)', 'Blockchain', 'Other']
t = df.groupby(['tag']).count().sort_values(by=['timestamp'], ascending = False).rename(columns={'timestamp':'count_'}).reset_index()#data clean up 
s = t[['tag', 'count_']].loc[t.count_>100]
values_to_map = set(s.tag.unique()) - set(current_tag_list)
list(values_to_map)

In [ ]:
#set up mapping 
mapping_dict = {'Analytics': "Data", 
                'Operations': "DevOps",
                'Embedded Systems': "Full Stack",
                'Validation': "Testing (SDET)",
                'Verification': "Testing (SDET)",
                'Analog': "Testing (SDET)", 
                'User Experience (UX)': "Web Development (Front-End)", 
                'Infrastructure': "Distributed Systems (Back-End)", 
                'Sales': "Other",
                'Hardware': "Full Stack",
                'Growth': "Other",
                'ASIC Design': "Web Development (Front-End)",
                'Enterprise': "Other",
                'Product': "Distributed Systems (Back-End)",
                'Technical': "DevOps",
                'SoC Design': "Web Development (Front-End)", 
                'Design': "Web Development (Front-End)",
                'Consumer': "Other",
                'General': "Other",
                'Cloud': "DevOps",
                'Embedded' : "Full Stack", 
                'Analytic': "Data"}

df['tag_clean'] = df['tag'].map(mapping_dict)
df[df.tag_clean.isin(current_tag_list)].head()

In [ ]:
df['tag_clean_2'] = np.nan
df['tag_clean_2'] = df.tag_clean_2.fillna(df.tag_clean).fillna(df.tag)
df.head()

In [ ]:
df.drop(columns = ['tag','tag_clean'], inplace = True)

## Impute Gender

In [ ]:
#set up salary groups
def group_data(x):
    val = abs(x)
    if x <= 50000: return "a. <50k"
    elif x<= 100000: return "b. 50-100k"
    elif x<= 150000: return "c. 100-150k"
    elif x<=200000: return "d. 150-200k"
    elif x<=250000: return "e. 200-250k"
    elif x<=300000: return "f. 250-300k"
    elif x<=350000: return "g. 300-350k"
    elif x<=400000: return "h. 350-400k"
    elif x<=450000: return "i. 400-450k"
    elif x<=500000: return "j. 450-500k"
    elif x<=1000000: return "k. 500k-1m"
    else: return "l. >1m"
    
df['salarygroup'] = df['totalyearlycompensation'].apply(group_data)

In [ ]:
t = df.groupby(['salarygroup']).count().rename(columns={'title':'count'}).reset_index()#data clean up 
sns.set(rc={'figure.figsize':(15,5)})
ax = sns.barplot(x="salarygroup", y="count", data=t)
ax.tick_params(axis='x', rotation=90)

In [ ]:
#find number of reportings for each salary group 
s = df[['salarygroup','gender', 'title']].dropna()
s.rename(columns = {'title':'count'}, inplace = True)
p = ['Male','Female','Other']
s = s[s.gender.isin(p)]
s = s.groupby(['salarygroup','gender']).count().rename({'title':'count_'})#.reset_index()#.groupby('salarygroup').sum()
s['gender'] = s.index.get_level_values(1)
s.droplevel(level=-1)
s = s.unstack()['count']
s.head()

In [ ]:
#find ratio of breakdown by salary group 
s["sum"] = s.sum(axis=1)
s['F_ratio'] = 100*(s['Female']/s['sum'])
s['M_ratio'] = 100*(s['Male']/s['sum'])
s['O_ratio'] = 100*(s['Other']/s['sum'])
s = s.fillna(0)
s = s[['F_ratio', 'M_ratio', 'O_ratio']]
df = df.copy()
s.head()

In [ ]:
#shuffle records in the dataframe 
df = df.iloc[np.random.permutation(len(df))]

In [ ]:
#assign missing records a gender based on salary ratio 
df['imputed_gender'] = None 


def impute_gender(mapping_df, data):
    
    #for each salary range 
    for salary_range in mapping_df.index:
        #print(salary_range)
        f_ratio, m_ratio, o_ratio = mapping_df.loc[salary_range, :]
        #print(f_ratio, m_ratio, o_ratio)
        #print(len)
        f_index = round(len(data.loc[data.salarygroup == salary_range])*(f_ratio/100))
        m_index = f_index + round(len(data.loc[data.salarygroup == salary_range])*(m_ratio/100))
        o_index = m_index + round(len(data.loc[data.salarygroup == salary_range])*(o_ratio/100))
        #print(f_index, m_index, o_index)
        count = 0 
        for i, row in data.loc[data.salarygroup == salary_range].iterrows():                       
           # print(row)
            if count <= f_index:
                data.at[i, 'imputed_gender'] = 'Female'
            elif count < m_index:
                data.at[i, 'imputed_gender'] = 'Male'
            else:
                data.at[i, 'imputed_gender'] = 'Other'
            count += 1
            
    return data

new_df = impute_gender(s, df.loc[(df.gender.isna()) | (~df.gender.isin(['Female', 'Male', 'Other']))][['salarygroup', 'imputed_gender']])     
new_df.head() 

In [ ]:
#merge back with original dataframe 
df = pd.merge(df, new_df[['imputed_gender']], how = 'left', left_index=True, right_index=True).drop(columns = ['imputed_gender_x'])
df['gender_2'] = None 
df['gender_2'] = df.gender_2.fillna(df.imputed_gender_y).fillna(df.gender)



In [ ]:
#clean dataframe 
df = df.drop(columns = ['imputed_gender_y', 'gender', 'timestamp'])
df.rename(columns = {'tag_clean_2': 'tag', 'gender_2': 'gender'}, inplace = True)

In [ ]:
df.columns

In [ ]:
 df.to_csv('edu_classification.csv')